## Using pandas with Large Data Sets

https://www.dataquest.io/blog/pandas-big-data/

When we move to larger data (100MB to multiple GB), performance issues can make run times much longer, and cause code to fail entirely due to insufficient memory.

While tools like Spark can handle large data sets (100GB to multiple TB) ... they lack rich feature sets for high quality data cleaning, exploration, and analysis. For medium-sized data, we’re better off trying to get more out of pandas.

In this post, we’ll learn about memory usage with pandas, how to reduce a dataframe’s memory footprint by almost 90%, simply by selecting the appropriate data types for columns.